In [1]:
from config import gapi_key
import csv
import pandas as pd
import os
import requests
import gmaps

In [3]:
# Read in data from previous analysis
path = os.path.join('..', 'WeatherPy', 'weather.csv')

weather = pd.read_csv(path)

weather.head()

,Latitude,Longitude,City,Country,City Latitude,City Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,-50.444117,-121.309702,rikitea,pf,-23.119901,-134.970265,79.27,77.0,0.0,14.99
1,68.622294,-150.636659,college,us,30.555443,-88.173882,-10.16,76.0,90.0,2.84
2,43.304920,-153.017137,kodiak,us,57.790000,-152.407222,19.40,48.0,40.0,19.46
3,31.331092,-46.497224,torbay,ca,47.658078,-52.735524,27.55,79.0,90.0,17.22
4,62.609263,-14.221523,hofn,is,64.249703,-15.202008,35.73,57.0,8.0,36.33


## Create a Heat Map
Create a heat map that displays the humidity for every city from the part I of the homework.

In [4]:
gmaps.configure(api_key=gapi_key)

In [5]:
locations = weather[["City Latitude", "City Longitude"]].astype(float)
humidity = weather["Humidity (%)"]

In [6]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Find ideal places to vacation

In [7]:
# Narrow down the DataFrame to find ideal weather conditions

temp = weather['Temperature (F)']
humid = weather['Humidity (%)']
clouds = weather['Cloudiness (%)']
wind = weather['Wind Speed (mph)']

ideal = weather[(temp <= 90) & (temp >= 70) & (humid <= 50) & (clouds <= 25) & (wind <= 15)]
ideal

,Latitude,Longitude,City,Country,City Latitude,City Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
9,-75.178937,94.272253,busselton,au,-33.653158,115.345489,79.77,31.0,0.0,7.36
80,-33.192436,148.815331,orange,au,-33.283333,149.100000,81.81,48.0,0.0,1.99
99,-33.756213,146.350045,griffith,au,-34.280088,146.045531,74.48,28.0,0.0,12.75
227,-28.874064,107.316085,geraldton,au,-28.779653,114.614445,84.20,35.0,0.0,10.29
290,-32.689539,136.284685,whyalla,au,-33.037967,137.575311,71.60,40.0,0.0,9.17
362,-21.827279,18.673903,gobabis,na,-22.399018,18.973995,71.37,39.0,13.0,8.75
447,-40.502422,177.046670,otane,nz,-39.896584,176.628424,86.00,50.0,17.0,8.01
479,31.642256,-150.835362,kahului,us,20.889335,-156.472947,76.06,50.0,20.0,5.82
495,-35.744692,147.041617,albury,au,-36.073729,146.913542,71.69,28.0,0.0,10.29


In [8]:
import requests
import json

### Get hotel

In [9]:

# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

# For each row in ideal df, find the first hotel that appears in Google Places search

hotels_info = []

for index, row in ideal.iterrows():
    
    h_info = {}
    
    lat = ideal.loc[index, 'City Latitude']
    lng = ideal.loc[index, 'City Longitude']
    city = ideal.loc[index, 'City'].title()
    country = ideal.loc[index, 'Country'].upper()
    coordinates = f"{lat}, {lng}"

    # set up search variables
    target_coordinates = coordinates
    target_radius = 5000
    target_type = "lodging"

    # declare the variables in the parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": gapi_key,
    } 

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Run the request
    response = requests.get(base_url, params=params)
    r = response.json()
    try:
        # print(r['results'][0])
        name = r['results'][0]['name']
        h_info = {
            'name': name,
            "city": city,
            "country": country,
            'location': (lat, lng) 
        }
        hotels_info.append(h_info)
    except:       
        print(coordinates)
        continue

print(hotels_info)

[{'name': 'Observatory Guest House', 'city': 'Busselton', 'country': 'AU', 'location': (-33.653158399999995, 115.3454892)}, {'name': 'Summer East Serviced Apartments Orange NSW', 'city': 'Orange', 'country': 'AU', 'location': (-33.283333, 149.1)}, {'name': 'Econo Lodge Griffith Motor Inn', 'city': 'Griffith', 'country': 'AU', 'location': (-34.280088299999996, 146.04553130000002)}, {'name': 'Hospitality Geraldton', 'city': 'Geraldton', 'country': 'AU', 'location': (-28.7796535, 114.6144452)}, {'name': 'Foreshore Motor Inn', 'city': 'Whyalla', 'country': 'AU', 'location': (-33.0379671, 137.5753107)}, {'name': 'Digby', 'city': 'Gobabis', 'country': 'NA', 'location': (-22.399017899999997, 18.9739954)}, {'name': 'Ludlow Farmstay', 'city': 'Otane', 'country': 'NZ', 'location': (-39.896583500000006, 176.62842369999998)}, {'name': 'Maui Beach Hotel', 'city': 'Kahului', 'country': 'US', 'location': (20.889335100000004, -156.4729469)}, {'name': 'Comfort Inn & Suites Georgian', 'city': 'Albury', 

In [10]:
hotel_locations = [hotel['location'] for hotel in hotels_info]
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
pop_ups = [info_box_template.format(**hotel) for hotel in hotels_info]

marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=pop_ups)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig = gmaps.figure()
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))